## Supabase: Backend como Servicio para Desarrollo Web en el Cliente

Supabase es un Backend como Servicio (BaaS) que proporciona un conjunto de herramientas para desarrollar aplicaciones web sin la necesidad de implementar un backend completo. Este servicio incluye características como SDK, analíticas, autenticación, bases de datos, almacenamiento, dashboard, WebSockets, entre otros.

### Características Principales

#### Base de Datos PostgreSQL
Supabase utiliza PostgreSQL como su motor de base de datos, permitiendo a los desarrolladores aprovechar todas las capacidades de esta robusta base de datos relacional.

#### API y SDK
Ofrece varios tipos de APIs y un SDK que facilita la interacción con el backend desde el cliente. También soporta el almacenamiento de archivos estáticos y permite la ejecución de funciones definidas por el usuario a través de RPC (Remote Procedure Call).

### Gestión de Tablas y API

Cuando se crea una tabla en Supabase, se genera automáticamente una API para interactuar con ella. Puedes ver la API generada en el SDK o en Bash. Aquí utilizaremos ejemplos en Bash para evitar la instalación del SDK.

```bash
curl 'https://bqhnvwfovmcxrqrsmfxr.supabase.co/rest/v1/graphs?select=id' \
-H "apikey: tu_apikey" \
-H "Authorization: Bearer tu_token"
```

Este comando `curl` realiza una petición GET a la tabla `graphs` para obtener el campo `id`.

En JavaScript, se puede hacer una solicitud similar usando `fetch`:



In [1]:

fetch('https://bqhnvwfovmcxrqrsmfxr.supabase.co/rest/v1/graphs', {
    method: 'GET',
    headers: {
        "apiKey": "tu_apikey",
        "Authorization": "Bearer tu_token"
    }
})


Promise { <pending> }



### Almacenamiento de JSON

En Supabase, el primer nivel de los objetos se guarda como columnas en la tabla. Si los objetos tienen subniveles o se realizan consultas a otras tablas, se guardan en formato JSON como texto.

### Autenticación

Las tablas públicas son accesibles con la apiKey pública. Sin embargo, al estar en el cliente, cualquiera podría ver esta apiKey en el código. Para proteger tablas privadas, se deben crear usuarios y aplicar métodos de autenticación y autorización. 

#### Activación de RLS (Row Level Security)

Primero, es necesario activar RLS en las tablas y crear políticas para controlar el acceso. Por ejemplo, podemos permitir que solo los usuarios autenticados puedan leer y escribir en una tabla.

#### Autenticación por Email y Contraseña

Podemos invitar usuarios a través de la interfaz o crear un formulario de inscripción. En cualquier caso, es necesario que el usuario confirme su email.

En el caso de la invitación, la redirección por defecto es a `localhost:3000`, pero esto se puede modificar. Una vez confirmado, el usuario recibirá un token que se puede probar en [jwt.io](https://jwt.io). Este token se usará en la cabecera `Authorization: Bearer`.

### Registro de Usuarios

En la documentación autogenerada de la API se explica qué datos enviar en Bash para registrar usuarios. Al iniciar sesión, nos quedaremos con el atributo `access_token` de la respuesta para utilizarlo en futuras peticiones.

### Gestión del Tiempo de Sesión

Después de iniciar sesión, guardamos el token en `LocalStorage` para realizar peticiones. Cada inicio de sesión retorna un tiempo de expiración (`expires_in` y `expires_at`). Podemos calcular la fecha de expiración y evitar confiar en el token después de esta fecha.

### Gestión de Perfiles de Usuario

Es común que los usuarios necesiten gestionar información personal como avatar, dirección, teléfono, etc. Para ello, Supabase proporciona un script SQL que crea una tabla de perfiles de usuario y asocia la información correspondiente.

#### Ejemplo de Manejo de Perfiles

Cuando un usuario inicia sesión, obtenemos sus datos de perfil y los almacenamos en `LocalStorage`:

```javascript
let dataLogin = await loginSupabase(email, password);
localStorage.setItem("access_token", dataLogin.access_token);
localStorage.setItem("uid", dataLogin.user.id);
localStorage.setItem("mail", dataLogin.user.email);
let dataProfile = await getData(`profiles?id=eq.${dataLogin.user.id}`, dataLogin.access_token);
dataProfile = dataProfile[0];
localStorage.setItem('dataProfile', JSON.stringify(dataProfile));
```

### Gestión de Avatares

Los avatares se guardan en "Storage". Es necesario crear un bucket con permisos de escritura y actualización para los usuarios autenticados. Para subir imágenes, se realiza un POST con un `FormData` que contiene la imagen.

#### Ejemplo de Subida de Imágenes

```javascript
async function updateProfile(profile) {
    let access_token = localStorage.getItem('access_token');
    let uid = localStorage.getItem('uid');
    let formImg = new FormData();
    formImg.append("avatar", profile.avatar, 'avatarProfile.png');
    let avatarResponse = await fileRequest(`/storage/v1/object/avatars/avatar${uid}.png`, formImg, access_token);
    profile.avatar_url = avatarResponse.urlAvatar;
    delete profile.avatar;
    let responseUpdate = await updateData(`profiles?id=eq.${uid}&select=*`, access_token, profile);
}
```

#### Subida de una Imagen a Storage

```javascript
const headersFile = {
    "apiKey": SUPABASE_KEY,
    "Authorization": `Bearer ${token}`,
    "x-upsert": true
};
let response = await fetch(`${urlBase}${url}`, {
    method: 'POST',
    headers: headersFile,
    body: formImg
});
if (response.status >= 200 && response.status <= 300) {
    if (response.headers.get("content-type")) {
        let datos = await response.json();
        datos.urlAvatar = `${urlBase}${url}`;
        return datos;
    }
    return {};
}
```

### Descarga de una Imagen

Si no se necesita autorización, se puede utilizar un bucket público y la URL proporcionada. Si se requiere autorización, se debe enviar la API key y el token.

```javascript
const headersFile = {
    "apiKey": SUPABASE_KEY,
    "Authorization": `Bearer ${token}`
};
let response = await fetch(`${url}`, {
    method: 'GET',
    headers: headersFile
});
if (response.status >= 200 && response.status <= 300) {
    if (response.headers.get("content-type")) {
        let datos = await response.blob();
        return datos;
    }
    return {};
}
let avatar_url = responseGet[0].avatar_url;
responseGet[0].avatar_blob = URL.createObjectURL(await getFileRequest(avatar_url, access_token));
```

### Otras Funcionalidades

#### Cantidad (Count)

Para obtener la cantidad de registros, se usa el método HEAD y la preferencia `count=exact`. El resultado se encuentra en el header `content-range`.

#### Consultas con OR Lógico

En la URL se puede usar la consulta `select=*&or=(id.eq.464,id.eq.466)`.

### WebSocket

Supabase ofrece una función llamada Realtime para manejar WebSockets. Se debe agregar en la sección de Database > Replication las tablas que queremos monitorear. Es muy recomendable utilizar las librerías proporcionadas por Supabase para esto.

#### Ejemplo de Comunicación "Broadcast"

Supabase permite la comunicación directa con otros usuarios o la notificación de cambios en las tablas mediante WebSockets.

### Ejemplo de Chat con Mensajes Guardados

Supabase facilita la implementación de un sistema de chat donde los mensajes se guardan en la base de datos. Puedes modificar el chat para mantener la sesión del usuario en cookies.

```javascript
// Código ejemplo de comunicación "broadcast" o chat
```

Supabase es una poderosa herramienta que facilita el desarrollo de aplicaciones web del lado del cliente, proporcionando un backend completo y fácil de usar. Sus características avanzadas de autenticación, almacenamiento, y tiempo real lo convierten en una opción robusta para desarrolladores que buscan una solución BaaS eficiente.